In [1]:
# ==================================================
# Azure Blob Storage Demo
# ==================================================
import sys, os
import json

sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("."))

from app.clients.blob_storage import BlobStorageClient


# Initialisierung
client = BlobStorageClient()

# Beispieldaten:
data = {
    "message": "Testinhalt - Azure Blob Storage funktioniert.",
    "source": "local_demo_script"
}
# Testdatei Erzeugung
blob_name = "test_upload.json"

# Datei hochladen
upload_ok = client.upload_file(data=data, blob_name=blob_name)
print("Upload erfolgreich: ", upload_ok)

# Dateien herunterladen
downloaded = client.download_file(blob_name)
print ("/nDownload erfolgreich: ", downloaded is not None)

# Lokalen Inhalt Überprüfung
if downloaded is not None:
    content = json.loads(downloaded.decode("utf-8"))
    print("\nInhalt der heruntergeladenen Dateien: ")
    print(content)

Upload erfolgreich:  True
/nDownload erfolgreich:  True

Inhalt der heruntergeladenen Dateien: 
{'message': 'Testinhalt - Azure Blob Storage funktioniert.', 'source': 'local_demo_script'}


In [3]:
# =========================================================
# Löschung vom bestehenden Index
# =========================================================
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchableField

from app.config import AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_KEY, AZURE_SEARCH_INDEX


index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)

# Wenn alter Index vorhanden ist, es wird zuerst gelöscht
try:
    index_client.delete_index(AZURE_SEARCH_INDEX)
    print(f"Alter Index ({AZURE_SEARCH_INDEX}) wurde gelöscht.")
except Exception:
    pass



Alter Index (documents) wurde gelöscht.


In [4]:
# ===========================================================
# Erstellung eines neuen Indexes
# ============================================================
fields = [
    SimpleField(name="id", type="Edm.String", key=True),
    SearchableField(name="title", type="Edm.String", retrievable=True),
    SearchableField(name="content", type="Edm.String", retrievable=True)
]

index = SearchIndex(name=AZURE_SEARCH_INDEX, fields=fields)

try:
    index_client.create_index(index)
    print(f"Index '{AZURE_SEARCH_INDEX}' erfolgreich initialisiert.")
except Exception as e:
    print(f"Fehler ist bei der Erstellung des Indexes aufgetreten: {e}")

Index 'documents' erfolgreich initialisiert.


In [5]:
# ========================================================
# Das Hochladen von Beispieldokumenten
# ========================================================
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential


search_client = SearchClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)

docs = [
     {"id": "1", "title": "Azure Cognitive Search Test", "content": "Dies ist ein Testdokument."},
    {"id": "2", "title": "Künstliche Intelligenz", "content": "Azure Search ermöglicht KI-basierte Suche."},
    {"id": "3", "title": "Cloud Computing", "content": "Speicherung und Verarbeitung in der Cloud."},
    {"id": "4", "title": "Python Programmierung", "content": "Python ist eine beliebte Programmiersprache."},
    {"id": "5", "title": "Azure Einsatz", "content": "Azure könnte tatsächlich bei Firmen auf verschiedenen Wegen genutzt werden. In erster Linie ist es hervorzuheben, dass Azure Cognitive Services einen großen Mehrwert bei zahlreichen heikligen Aufgaben verschaffen kann."},
    {"id": "6", "title": "FastAPI", "content": "Erstellung von APIs mit FastAPI."}
]

search_client.upload_documents(documents=docs)
print("Dokumente erfolgreich hochgeladen.")

Dokumente erfolgreich hochgeladen.


In [6]:
# ========================================================
# Cognitive Search Demo
# =========================================================
from app.clients.cognitive_search import CoginitiveSearchClient
from app.config import AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_KEY, AZURE_SEARCH_INDEX

# Initialisierung des Clients
try:
    search_client = CoginitiveSearchClient()
    print("Cognitive Search Client wurde erfolgreich initialisert.")
except Exception as e:
    print("Fehler bei der Initialisierung: {e}")

query = "Azure"

try:
    results = search_client.search_documents(query=query, top=5)
    print(f"Gefundene Dokumente: {len(results)}")
    for doc in results:
        print(doc)
except Exception as e:
    print(f"Fehler bei der Suche: {e}")

Cognitive Search Client wurde erfolgreich initialisert.
Gefundene Dokumente: 3
{'content': 'Azure könnte tatsächlich bei Firmen auf verschiedenen Wegen genutzt werden. In erster Linie ist es hervorzuheben, dass Azure Cognitive Services einen großen Mehrwert bei zahlreichen heikligen Aufgaben verschaffen kann.', 'title': 'Azure Einsatz', 'id': '5', '@search.score': 1.6750907, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'content': 'Azure Search ermöglicht KI-basierte Suche.', 'title': 'Künstliche Intelligenz', 'id': '2', '@search.score': 1.133612, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'content': 'Dies ist ein Testdokument.', 'title': 'Azure Cognitive Search Test', 'id': '1', '@search.score': 0.7648603, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}


In [7]:
# ========================================
# OpenAI Modell Aufruf Demo
# ========================================
import sys
import os 

sys.path.append(os.path.abspath(".."))

from app.clients.openai_client import OpenAIClientWrapper

client = OpenAIClientWrapper()
response = client.generate_answer("Was ist Azure OpenAI?", max_tokens=1000)
print(response)

Azure OpenAI ist ein Service von Microsoft, der es ermöglicht, die KI-Modelle von OpenAI, wie GPT-3 und andere, in Azure-Umgebungen zu integrieren. Mit Azure OpenAI können Unternehmen und Entwickler auf fortschrittliche Sprachverarbeitungs- und KI-Technologien zugreifen, um Anwendungen zu erstellen, die natürliche Sprache verstehen und generieren können.

Einige wichtige Aspekte von Azure OpenAI sind:

1. **API-Zugriff**: Benutzer können über eine API auf die KI-Modelle zugreifen, um Text zu generieren, Fragen zu beantworten, Texte zu analysieren und vieles mehr.

2. **Integration mit Azure-Diensten**: Azure OpenAI kann nahtlos mit anderen Azure-Diensten, wie Azure Machine Learning, Azure Cognitive Services und Azure Functions, integriert werden, um komplexe Lösungen zu entwickeln.

3. **Sicherheit und Compliance**: Microsoft bietet Sicherheitsfunktionen und Einhaltung von Vorschriften, die in vielen Unternehmensumgebungen erforderlich sind, und ermöglicht es Benutzern, KI-Anwendungen 

In [8]:
# ==============================================================
# RAG Pipeline (Azure KI Suche) mit dem Modell Integration: Demo
# ==============================================================
import os 
import sys

sys.path.append(os.path.abspath(".."))

from app.clients.cognitive_search import CoginitiveSearchClient
from app.clients.openai_client import OpenAIClientWrapper


# Initialisierung von Clients
search_client = CoginitiveSearchClient()
openai_client = OpenAIClientWrapper()

# Suchanfrage in Azure KI Suche
query = "Azure"
docs = search_client.search_documents(query=query, top=3)

# Antwort von GPT basierend auf Dokumenten wird erzeugt
prompt = "Erkläre mir, wie Azure in Unternehmen eingesetzt werden kann."
response = openai_client.generate_answer(prompt, context_docs=docs)
print(response)

Azure kann in Unternehmen auf vielfältige Weise eingesetzt werden, um verschiedene Anforderungen und Herausforderungen zu adressieren. Hier sind einige der wesentlichen Aspekte:

1. **Cognitive Services**: Azure bietet eine Reihe von Cognitive Services, die Unternehmen dabei unterstützen, komplexe Aufgaben zu automatisieren und wertvolle Einblicke aus ihren Daten zu gewinnen. Diese Dienste ermöglichen Funktionen wie Bilderkennung, Sprachverarbeitung und sentimentale Analyse, die in Anwendungen integriert werden können, um die Benutzererfahrung zu verbessern und Geschäftsentscheidungen zu optimieren.

2. **Azure Search**: Azure Search ermöglicht es Unternehmen, KI-basierte Suchfunktionen zu implementieren. Dies ist besonders wertvoll für Organisationen, die große Datenmengen verwalten und ihren Nutzern eine effiziente und relevante Sucherfahrung bieten möchten. Mit Azure Search können Unternehmen Benutzeranfragen analysieren und personalisierte Ergebnisse zurückgeben, wodurch die Auffin

In [1]:
# =================================================
# Azure Open AI Chatservice Demo
# =================================================
import os 
import sys

sys.path.append(os.path.abspath(".."))

from app.services.chat_service import ChatService


chat_service = ChatService()
user_id = "testuser"
question = "Bist du da?"

answer = chat_service.process_query(question=question, user_id=user_id)
print(f"The Antwort ist: {answer}")

Die upload_file Funktion wurde ohne blob_name aufgerufen.


The Antwort ist: Ja, ich bin da! Wie kann ich Ihnen helfen?
